In [1]:
import numpy as np
import matplotlib.pyplot as plt
from matplotlib import colors
import os
import fnmatch

In [2]:
grid =[]
gofn ={}
hofn ={}
parent_dict = {}
visited =[]
maze_size = 0
folder_counter=0

agent_grid = []
movements_list = []

In [3]:
def populateMatrix(n,p):
    """
    This function returns randomly populated maze
    p: probabillity density of each cell
    n: size of the maze
    """
    global grid
    np_array = np.random.rand(n*n)
    block = lambda x: 0 if x>=p else 1
    vectorized_block = np.vectorize(block)
    np_array = vectorized_block(np_array).reshape(n,n)
    np_array[0,0]=0
    np_array[n-1,n-1] =0
    grid = np.asmatrix(np_array)

In [4]:
def initialize(n,p):
    global agent_grid
    global maze_size
    global gofn
    global hofn
    global parent_dict
    global visited
    
    gofn ={}
    hofn = {}
    parent_dict = {}
    maze_size = n
    visited = []
    populateMatrix(n,p)
    agent_grid = np.asmatrix(np.zeros(n*n, dtype=int).reshape(n,n))

In [5]:
def manhattan_distance_calc(x1, y1, x2, y2):
    """
    This function returns manhattan distance for every cell
    i: row value
    j: column value
    n: size of the maze
    """
    return abs(x1 - x2) + abs(y1 - y2)


In [6]:
def get_children(x, y, n,visited, matrix):
    """
    This function is used to return children of a particular node
    x: row value of current node
    y: column value of current node
    n: size of maze
    gofn: g(n) value of current node
    visited_list: list of nodes that are already visited
    matrix: maze
    agent_matrix: matrix of the agent that holds the information of updated environment
    """
    allChildren = [(x+1,y),#down
                   (x-1,y),#up
                   (x,y+1),#right
                   (x,y-1)]#left
    
    if x+1 > n-1:
        allChildren.remove((x+1,y))
    if x-1 < 0:
        allChildren.remove((x-1,y))
    if y+1 > n-1:
        allChildren.remove((x,y+1))
    if y-1 < 0:
        allChildren.remove((x,y-1))
        
    allChildren = [node for node in allChildren if node not in visited]
    
    allChildren = [node for node in allChildren if matrix[node]!=1]
    

    return allChildren

In [7]:
def set_parent(children,parent):
    global parent_dict
    for child in children:
        parent_dict[child] = parent

In [8]:
def update_gofn(children, parent):
    global gofn
    
    for child in children:
        gofn[child] = gofn[parent] +1

In [9]:
def update_hofn(children, goal):
    global hofn
    
    for child in children:
        hofn[child] = manhattan_distance_calc(child[0],child[1],goal[0],goal[1])

In [10]:
def get_path_to_goal(source,node):
    global parent_dict
    
    path =[]
    while(node!=source):
        path.append(node)
        node = parent_dict[node]
    path.append(source)
    return path

In [11]:
def display2(trajectory_path,matrix):
    """
    This function is used to display maze with traversed path(highlighted)
    trajectory_path: path traversed by the agent so far
    matrix: maze of the problem
    """
    temp = matrix.copy()
    for path in trajectory_path:
        temp[path[:2]] = 2
    colormap = colors.ListedColormap(["white","black","green"])
    plt.imshow(temp, cmap=colormap)
    plt.show()

In [12]:
def display_input_map(matrix):
    """
    This function is used to display maze with traversed path(highlighted)
    trajectory_path: path traversed by the agent so far
    matrix: maze of the problem
    """
    temp = matrix.copy()
    colormap = colors.ListedColormap(["green","black","white","blue"])
    plt.imshow(temp, cmap=colormap)
    plt.show()

In [13]:
def astar(source, goal, matrix):
    global maze_size
    global gofn
    global hofn
    global parent_dict
    global visited
    
    visited = []
    hofn = {}
    gofn = {}
    parent_dict = {}
    Q = {}
    Q[source] = 0
    parent_dict[source] = source
    gofn[source] = 0
    goal_reached = False
    
    while not goal_reached and Q:
        Q = dict(sorted(Q.items(), key=lambda item: item[1]))
        current = list(Q)[0]
        Q.pop(current)
        visited.append(current)
        
        if(current == goal):
            goal_reached = True
            return get_path_to_goal(source,current)[::-1]
        
        children = get_children(current[0],current[1],maze_size,visited, matrix)
        set_parent(children, current)
        update_gofn(children,current)
        update_hofn(children, goal)
        for child in children:
            Q[child] = gofn[child] + hofn[child]
    return []

In [14]:
# initialize(4,0.3)
# print(maze_size)
# plt.spy(grid)
# path = astar((0,0),(3,3),agent_grid)
# path

In [15]:
"""
1 free visited
2 blocked
3 unvisited
4 current position
"""

def encode_grid(matrix, vis):
    mat = matrix.copy()
    mat[mat==1] = 2
    for pos in vis:
        mat[pos] = 1
    mat[mat==0] = 3
    return mat
    

In [16]:
def repeated_astar():
    """
    This function is used to call a_star repeatedly to get the shortest path from start to the goal node
    """
    
    global grid
    global maze_size
    global agent_grid
    global block_unblock_matrix

    matrix_snapshots = []
    positions = []
    goal_reached = False
    goal = (maze_size-1,maze_size-1)
    path = []
    visited =[]
    agent_grid = np.asmatrix(np.zeros(maze_size*maze_size, dtype=int).reshape(maze_size,maze_size))
    
    final_path = [(0,0)]
    
    
    while not goal_reached:
        path = astar(final_path[-1],goal,agent_grid)
        if(len(path) == 0):
            return ([],[],[])

        for node in path:
            matrix_snapshots.append(encode_grid(agent_grid, visited))
#             print("start\n",matrix_snapshots)
#             print(agent_grid)
            if grid[node] == 0:
                visited.append(node)
                if node not in final_path:
                    final_path.append(node)
#                     trajectory_length += 1
                if node == goal:
                    goal_reached = True
                    positions.append(node)
                    matrix_snapshots[-1][node] = 4 
#                     print("goal reached")
                    break
            else:
                agent_grid[node] = 1
                positions.append(node)
                matrix_snapshots[-1][node] = 4 
#                 print(node)
                break
            positions.append(node)
            matrix_snapshots[-1][node] = 4 
#             print(node)
#             
    return (final_path,matrix_snapshots,positions)

In [17]:
# initialize(4,0.3)
# plt.spy(grid)

In [18]:
# (path,snaps,pos) = repeated_astar()

In [19]:
# display2(path,grid)

In [20]:
# len(pos)
# path1 = astar((0,0),(3,3),grid)

In [21]:
def movement_of_agent(p1,p2):

        x1=p1[0]
        y1=p1[1]
        
        x2=p2[0]
        y2=p2[1]
        diffx = x2 - x1
        diffy = y2 - y1
        move_list = []     #[up, down, left, right]
        
        if diffx==-1 and diffy==0: #up
            move_list = [1,0,0,0] 
            return move_list
        elif diffx==1 and diffy==0: # down
            move_list = [0,1,0,0]
            return move_list
        elif diffx==0 and diffy==-1: #left
            move_list = [0,0,1,0]
            return move_list
        elif diffx==0 and diffy==1: #right
            move_list = [0,0,0,1]
            return move_list
        else:
            return "Error!!"

In [22]:
def get_data(maze_size,p):
    global grid
    
    final_data = []
    initialize(maze_size,p)
    # plt.spy(grid)
    path,snaps,pos = repeated_astar()
#     for i,j in zip(snaps,pos):
#         print(i,j)
    snaps = snaps[:-1]
    if(len(path)!=0):
        for i in range(len(snaps)):
            if i-1>=0 and pos[i+1] == pos[i-1]:
#                 print("skipped for: ",i,"where i+1:",pos[i+1]," i-1:",pos[i-1])
                continue
            movement = movement_of_agent(pos[i],pos[i+1])
            final_data.append((np.array(snaps[i].copy().flatten()),np.array(movement)))
#             final_data.append((snaps[i].copy(),np.array(movement)))
    return final_data

In [23]:
# temp_data = get_data(5,0.3)

In [24]:
# temp_data

In [25]:
def save_dataset(maze_size,p,iterations):
    global folder_counter
    counter = 0
    file_counter = 0
    while counter!= iterations:
        dataset = get_data(maze_size,p)
        if len(dataset)!=0:
            f = open("./dataset"+str(folder_counter)+"/data"+str(file_counter)+".npy", 'wb')
            counter += 1
            file_counter+=1
            for item in dataset:
                np.save(f,item[0])
                np.save(f,item[1])
            f.close()
    print("folder: ",folder_counter," done.")
    folder_counter+=1
    

In [26]:
# save_dataset(4,0.3,10000)

In [27]:
# save_dataset(4,0.3,10000)

In [28]:
# save_dataset(4,0.3,10000)

In [29]:
# save_dataset(4,0.3,10000)

In [30]:
# save_dataset(4,0.3,10000)

In [31]:
# save_dataset(4,0.3,10000)

In [32]:
# save_dataset(4,0.3,10000)

In [33]:
# save_dataset(4,0.3,10000)

In [34]:
# save_dataset(4,0.3,10000)

In [35]:
# save_dataset(4,0.3,10000)

In [36]:
def load_data(path_to_file,maze_size):
    X = []
    y = []
    f = open(path_to_file,'rb')
    
    try:
        while True:
            X.append(np.load(f).reshape(maze_size,maze_size))
            y.append(np.load(f).tolist())
    except:
            # print("eof")
            pass
    return X,y
    

In [37]:
def read_dataset(maze_size):
    final_X = []
    final_y = []
    for f in os.listdir("./dataset/"):
        if fnmatch.fnmatch(f,'data*'):
            X,y = load_data("./dataset/"+f,maze_size)
            print(y)
            final_X += X
            final_y += y
    
    return final_X,final_y

In [38]:
# maze_size=4
# X,y = read_dataset(maze_size)

In [39]:
# X[:3]

In [40]:
def display_collected_data(X,y):
    for i in range(len(X)):
        display_input_map(np.asmatrix(X[i]))
        if y[i][0]==1:
            print("Up")
        elif y[i][1]==1:
            print("down")
        elif y[i][2]==1:
            print("left")
        elif y[i][3]==1:
            print("right")

In [41]:
# display_collected_data(X,y)

In [42]:
# folder_counter=0